In [56]:
import os
import pandas as pd
import PyPDF2
from IPython.display import display, Markdown
import re

pd.set_option('display.max_colwidth', 40)

In [57]:
def read_pdf(file_path):
    try:
        with open(file_path, 'rb') as f:
            text = "\n".join(
                page.extract_text() or "" 
                for page in PyPDF2.PdfReader(f).pages
            )
        return text.strip()
    except Exception as e:
        print(f"Ошибка в файле {os.path.basename(file_path)}: {str(e)}")
        return ""

In [58]:
def parse_habr_article(text):
    if not text:
        return {}
    
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    result = {
        'title': 'Без названия',
        'author': 'Неизвестен',
        'date': '',
        'tags': [],
        'hubs': []
    }
    
    title_lines = []
    for line in lines:
        if not re.fullmatch(r'^\d+\.\d+$', line):
            title_lines.append(line)
            if any(x in line.lower() for x in ['неизвестен', 'нец', 'автор', '@']):
                break
    
    if title_lines:
        result['title'] = ' '.join(title_lines).strip()
        result['title'] = re.sub(r'^\d+\.\d+\s+', '', result['title'])
    
    for i, line in enumerate(lines):
        if 'неизвестен' in line.lower() or 'нец' in line.lower():
            result['author'] = 'Неизвестен'
            for date_line in lines[i+1:i+3]:
                if any(x in date_line.lower() for x in ['час', 'день', 'назад']):
                    result['date'] = date_line.strip()
                    break
            break
    
    full_text = ' '.join(lines)
    
    # Теги (учитываем разные варианты написания)
    tags_match = re.search(r'(Теги|Тети|Tags)[:\s]*([^ХабыHubs\n]+)', full_text, re.IGNORECASE)
    if tags_match:
        result['tags'] = [t.strip() for t in re.split(r'[,;]\s*', tags_match.group(2)) if t.strip()]
    
    hubs_match = re.search(r'(Хабы|Hubs|Habr)[:\s]*([^\n]+)', full_text, re.IGNORECASE)
    if hubs_match:
        result['hubs'] = [h.strip() for h in re.split(r'[,;]\s*', hubs_match.group(2)) if h.strip()]
    
    result['title'] = result['title'].split('Рейтинг')[0].strip()
    result['title'] = re.sub(r'\s+', ' ', result['title'])  
    
    return result

In [59]:
def process_folder(folder_path):
    
    data = []
    
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.pdf'):
            filepath = os.path.join(folder_path, filename)
            text = read_pdf(filepath)
            meta = parse_habr_article(text)
            
            data.append({
                'Файл': filename,
                'text_article': meta.get('title', ''),
                'Автор': meta.get('author', ''),
                'Дата': meta.get('date', ''),
                'Теги': ', '.join(meta.get('tags', [])),
                'Хабы': ', '.join(meta.get('hubs', []))
            })
    
    return pd.DataFrame(data)

In [60]:
folder_path = r'C:/Users/артем/Desktop/PDF2'  

print(f"Проверяемый путь: {repr(folder_path)}") 
print(f"Абсолютный путь: {os.path.abspath(folder_path)}")
print(f"Папка существует: {os.path.exists(folder_path)}")

if not os.path.exists(folder_path):
    display(Markdown("**Ошибка:** Папка не найдена!"))
    desktop_path = os.path.join('C:', 'Users', 'артем', 'Desktop')
    if os.path.exists(desktop_path):
        display(Markdown("**Содержимое рабочего стола:**"))
        try:
            display(os.listdir(desktop_path))
        except Exception as e:
            display(Markdown(f"Ошибка при чтении папки: {e}"))
else:
    df = process_folder(folder_path)
    
    display(Markdown("### Результаты обработки:"))
    display(df)
    
    csv_path = 'habr_results.csv'
    try:
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        display(Markdown(f"Данные сохранены в **{csv_path}** (UTF-8 CSV)"))
        
        excel_path = 'habr_results.xlsx'
        df.to_excel(excel_path, index=False)
        display(Markdown(f"Данные также сохранены в **{excel_path}** для совместимости"))
    except Exception as e:
        display(Markdown(f"**Ошибка при сохранении:** {e}"))

Проверяемый путь: 'C:/Users/артем/Desktop/PDF2'
Абсолютный путь: C:\Users\артем\Desktop\PDF2
Папка существует: True


### Результаты обработки:

,Файл,text_article,Автор,Дата,Теги,Хабы
0,"Cocoapods, Carthage, SPM как выбрать...",,Неизвестен,,й в,"Блог компании red_mad_robot, Разрабо..."
1,Deutsche Telekom и Perplexity объяви...,Еще до на чала MWC в Барсе лоне был ...,Неизвестен,,см,"Искусственный инт еллек т, Смар тфон..."
2,imgonline-com-ua-site2pdfB28eKGLy6Ej...,,Неизвестен,,"взлом, соцсети, o","Блог компании Cloud4Y, Информационна..."
3,OSINT & Hacking — как работает фишин...,,Неизвестен,,"взлом, соцсети, o","Блог компании Cloud4Y, Информационна..."
4,Yandex N.V. запретили в течение пяти...,Нидер ландск ая компания Y andex N.V...,Неизвестен,,"яндек с, yandex n.v ., р","Управ ление проек тами, Бизнес-мо де..."
5,Быстрое начало работы с Gitlab CICD.pdf,Оценка,Неизвестен,,"ci/cd, gitla","Блог компании Southbridge, Тестирова..."
6,Генеральный директор Mozilla покинул...,"Митче лл Бейк ер, гендирек тор Mozil...",Неизвестен,,"mozilla, mozilla fo","IT-компании, Управ ление перс онал о..."
7,За что безопасники будут гореть в ад...,Для прив лечения внимания расск ажу ...,Неизвестен,,сво,.com/ru/articles/888708/ 1/11 Из опы...
8,Заезжаем в Kotlin Multiplatform. Но ...,"Прив ет! Меня з овут Денис, я занима...",Неизвестен,,,
9,ИИ-агенты в Альфа-Банке_ нейросети с...,,Неизвестен,,,"Блог компании Альфа-Банк, Искусствен..."


Данные сохранены в **habr_results.csv** (UTF-8 CSV)

**Ошибка при сохранении:** RUVDS.com VDS/VPS-хостинг. Скидка 15% по коду HABR15 Пишет в хабы: Читальный зал ,  DIY или Сделай сам ,  Научно-популярное ,  Электроника для начинающих ,  Системное администрирование 72K Selectel IT-инфраструктура для бизнеса Пишет в хабы: Компьютерное железо ,  Гаджеты ,  IT-инфраструктура ,  IT-компании ,  Производство и разработка электроники 40K Timeweb Cloud Облачная платформа для разработчиков и бизнеса Пишет в хабы: Читальный зал ,  Научно-популярное ,  JavaScript ,  DIY или Сделай сам ,  История IT 1.1K OTUS Цифровые навыки от ведущих экспертов Пишет в хабы: Программирование ,  Java ,  Машинное обучение ,  Тестирование веб-сервисов ,  Python 103K Яндекс Как мы делаем Яндекс Добавить компанию Все потоки СТАТЬИ +64 НОВОСТИ +50 ХАБЫ АВТОРЫ cannot be used in worksheets.